## Deep Learning for Time Series 

Deep Learning is a subset of machine learning and excels when dealing with large and complex data, as it can extract complex features with minimal human involvement. Deep learning works well with structured and unstructured data and can be used in supervised, unsupervised and semi-supervised learning. 

This chapter focuses on using deep learning for time series forecasting - using different deep learning architectures suitable for sequential data such as time series data. There are different deep learning architectures for solving various problems: 

* Recurrent Neural Networks (RNNs)
* Long-Short Term Memory (LSTM)
* Gated Recurrent Unit (GRU)
* Convolutional Neural Networks (CNNs)
* Autoencoders
* Generative Adversarial Networks (GANs)

In [1]:
import pandas as pd 
import numpy as np 


In [2]:
class Standardize:
    def __init__(self, df, split=0.10):
        self.data = df 
        self.split = split 

    def split_data(self):
        n = int(len(self.data) * self.split)
        train, test = self.data.iloc[:-n], self.data.iloc[-n:]
        n = int(len(train) * self.split)
        train, val = train.iloc[:-n], train.iloc[-n:]
        assert len(test) + len(train) + len(val) == len(self.data)
        return train, test, val 
    
    def _transform(self, data):
        data_s = (data - self.mu)/self.sigma 
        return data_s 
    
    def fit_transform(self):
        train, test, val = self.split_data()
        self.mu, self.sigma = train.mean(), train.std()
        train_s = self._transform(train)
        test_s = self._transform(test)
        val_s = self._transform(val)
        return train_s, test_s, val_s 
    
    def inverse(self, data):
        return (data * self.sigma)+self.mu 
    
    def inverse_y(self, data):
        return (data * self.sigma[-1])+self.mu[-1]
    


In [3]:
def one_step_forecast(df, window):
    d = df.values 
    x = []
    n = len(df)
    idx = df.index[:-window]
    for start in range(n-window):
        end = start + window
        x.append(d[start:end])
    cols = [f'x_{i}' for i in range(1, window+1)]
    x = np.array(x).reshape(n-window, -1)
    y = df.iloc[window:].values
    df_xs = pd.DataFrame(x, columns=cols, index=idx)
    df_y = pd.DataFrame(y.reshape(-1), columns=['y'], index=idx)

    return pd.concat([df_y, df_xs], axis=1).dropna()

In [4]:
# Example 

import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from pathlib import Path 
import warnings
warnings.filterwarnings('ignore')
path = Path('../TimeSeriesAnalysisWithPythonCookbook/Data/')
energy = pd.read_csv(path.joinpath('energy_consumption.csv'), index_col='Month', parse_dates=True)
energy.columns = ['y']
energy.index.freq = 'MS'
en_df = one_step_forecast(energy, 10)

In [5]:
# Scale the data using the updated Standardize class 
scale_en = Standardize(en_df)

In [6]:
train_en, test_en, val_en = scale_en.fit_transform()

Deep Learning libraries can be broken down into either **low-level, high-level** or both. High-level libraries allow for quick prototyping and experimentation when testing various architectures, such as the case with Keras. A low-level library gives us more flexibility and control, but we will have to define more aspects of a model's architecture - PyTorch and Tensorflow are examples of low-level libraries. 

## Forecasting with an RNN using Keras 

RNNs initially entered the spotlight with NLP, as they were designed for sequential data, where past observations, such as words, have a strong influence on determining the next word in a sentence. This need for the artificial neural network to retain memory (hidden state) inspired the RNN architecture. Similarly, time series data is also sequential, and since past observations influece future observations, it also needs a network with memory. 

In RNNs, there is a feedback loop where the output of one node or neuron is fed back (the recursive part) as input, allowing the network to learn from a prior time step acting as a memory. 

In an RNN we have two outputs and two sets of weights: $W_{X}$ for the input and $W_{H}$ for the hidden state or memory



In [17]:
import tensorflow as tf 
#import keras 
from keras.models import Sequential
#keras.models.Sequential
#import tf.keras.models.Sequential 

# import Sequential 
#from tensorflow import keras 
# from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError 
# from keras.layers import Dense, SimpleRnn, Dropout

ModuleNotFoundError: No module named 'tensorflow.compat'

In [21]:
# Create the features_target_ts function that takes a dataset and returns an x split 
# (independent variables or features) and y split (dependent or target variables)

def features_target_ts(*args):
    y = [col.pop('y').values.reshape(-1,1) for col in args]
    x = [col.values.reshape(*col.shape, 1) for col in args]

    return *y, *x


In [22]:
# We can pass the train, test and validation sets to the features_target_ts function and
# it will return six splits

(y_train_en, y_val_en, y_test_en, x_train_en, x_val_en, x_test_en) = features_target_ts(train_en, val_en, test_en)


In [33]:
# Create the create_model function, which is used to construct the network's architecture.
# The Sequential class will sequentially add or stack the different layers in the order added,
# hence the name. We will implement the SimpleRNN architecture 

def create_model(train, units, dropout=0.2):
    model = tf.keras.Sequential()
    model.add(tf.keras.models.SimpleRNN(units=units, 
                        return_sequences=False,
                        input_shape=(train.shape[1], 
                                     train.shape[2])))
    model.add(tf.keras.metrics.Dropout(dropout))
    model.add(tf.keras.layers.Dense(1))
    return model 


The create_model function will return a Sequential object which contains the architecture (the layers and the configuration). We are adding a dropout layer that randomly drops some of the unity by setting them to zero. The frequency is set at 0.2 (20%), indicating the fraction of the input units to drop. return_sequence is set to False, indicating that only the last output is returned

Create the train_model_ts function, which takes as input the returned Sequential object (which we are calling model), and the training and validation sets. The function will compile and train the model on the training sets and use the validation sets for evaluation at each epoch, displaying the scores against the training and validation sets:

In [34]:
def train_model_ts(model, x_train, y_train, x_val, y_val, epochs=500, patience=12, batch_size=32):
    model.compile(optimizer="adam", 
                  loss='mean_squared_error', 
                  metrics=[RootMeanSquaredError(), 
                           MeanAbsoluteError()])
    
    es = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", 
        min_delta=0, 
        patience=patience)
    
    history = model.fit(x_train, y_train, 
                        shuffle=False, epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data = (x_val, y_val), 
                        callbacks=[es], verbose=1)
    
    return history


The function will return the history object, which is a Python dictionary that includes all the scores captured at each epoch for the training and validation sets



Create the plot_forecast function, which will take the model object to make a prediction (forecast) and print out the predicted values against the actual values (out-of-sample) in the test set. Additionally, the function takes the history dictionary to plot the model's performance during training, so we can visually evaluate the model for any signs of overfitting

In [35]:
def plot_forecast(model, x_test, y_test, index, history):
    fig, ax = plt.subplots(2, 1)
    (pd.Series(history.history['loss']).plot(style='k', alpha=0.5, title='Loss by Epoch',
           ax = ax[0], label='loss'))
    
    (pd.Series(history.history['val_loss']).plot(style='k', ax=ax[0], label='val_loss'))
    ax[0].legend()
    predicted = model.predict(x_test)
    pd.Series(y_test.reshape(-1), 
              index=index).plot(style='k--', alpha=0.5, ax=ax[1], title="Forecast vs Actual", label='actual')
    pd.Series(predicted.reshape(-1), 
              index=index).plot(style='k', label='Forecast', ax=ax[1])
    fig.tight_layout()
    ax[1].legend()
    plt.show()
    

The function displays two subplots - the first plot will contain the performance during training (training and validation loss) and the bottom chart will compare the forecast. 


In [36]:
# Use the create_model function to create the Sequential object

model_en_simpleRNN = create_model(x_train_en, units=32)

AttributeError: module 'tensorflow' has no attribute 'keras'